In [1]:
!pip install git+https://github.com/lesc-ufv/cad4u &> /dev/null
!git clone https://github.com/lesc-ufv/cad4u &> /dev/null
%load_ext plugin


Considere que desejamos gerar um array B através de um array A de mesmo tamanho, de modo que, cada elemento `B[i]` é dado pela soma do elemento `A[i]` mais os `r` elementos anteriores à posição `i` em A mais os `r` elementos posteriores, ou seja:
```
for(j=i-r; j<=i+r; j++)
    B[i]+=A[j]
```
O código dado a seguir já faz o cálculo do array B (armazenado em `b_cpu`) de forma sequencial. Escreva um kernel CUDA, bem como todo o restante do código necessário para executá-lo, de modo a permitir o cálculo de B de forma paralela.

As posições inválidas no início e final do array A devem ser simplesmente descartadas durante a soma, ou seja, para cálculo de `B[10]` com `r=32`, o resultado será a soma dos elementos de `A[0]` até `A[42]`.

Se atente para os seguinte requisitos:
* O cálculo dos elementos de B deve ser realizado de forma paralela;
* A memória compartilhada da GPU deve ser utilizada de forma a deixar a operação mais eficiente;
* O código deve tratar corretamente arrays grandes divididos em mais de um bloco.

Dicas:
* O código abaixo utiliza a constante `RANGE` para definir o valor de `r` e a constante `BLOCKSIZE` para definir o tamanho de cada bloco do grid;
* Por simplicidade, você pode considerar que o tamanho do array A será sempre múltiplo de `BLOCKSIZE`;
* Considere também que o tamanho de `r` será no mínimo igual a 0 e no máximo igual à metade de `BLOCKSIZE`, ou seja, o tamanho do array alocado na memória compartilhada será no máximo igual a `2*BLOCKSIZE`, e no mínimo igual ` BLOCKSIZE`.
* A estratégia utilizada pela versão sequencial de testar cada posição (para descobrir se é válida) não é ótima. Muitas comparações desnecessárias são realizadas ao calcular as posições intermediárias. Na versão paralela, é melhor preecher as posições inválidas com o valor 0 (zero) ao copiar os dados para a memória compartilhada.

In [2]:
%%gpu
#include<stdio.h>
#include<cuda.h>

#define BLOCKSIZE 1024  //threads por bloco
#define RANGE 256       //deslocamento utilizado para cálculo de B

void initArray(int *a, int n) {
    for (int i =0; i<n; ++i)
        a[i]=i;
}

void printArray(int *a, int n) {
    int maxPrint=20;
    for (int i =0; i< min(n,maxPrint); ++i)
        printf("%5d ",a[i]);
    if (n>maxPrint) printf("... (array truncado)");
    printf("\n");
}

bool checkArray(int *a, int *b, int n) {
    for (int i =0; i<n; ++i)
        if (a[i]!=b[i])
            return false;
    return true;
}

void execHost(int *A, int n, int *B) {
    for(int i=0; i<n; i++) {
        int soma=0;
        for(int j=i-RANGE; j<=i+RANGE; j++)
            if(j >= 0 && j<n)
                soma+=A[j];
        B[i]=soma;
    }
}

int main() {
    int N = 1<<20;              //tamanho dos arrays A e B
    int *a = new int[N];
    int *b_cpu = new int[N];    //array B calculado na CPU
    int *b_gpu = new int[N];    //array B calculado na GPU

    initArray(a,N);
    printf("    A: ");
    printArray(a,N);

    printf("B-GPU: ");
    printArray(b_gpu,N);

    printf("B-CPU: ");
    execHost(a, N, b_cpu);
    printArray(b_cpu,N);

    if (checkArray(b_gpu,b_cpu,N))
        printf("Resultados iguais\n");
    else
        printf("Resultados diferentes\n");

    free(a);
    free(b_cpu);
    free(b_gpu);
    return 0;
}

    A:     0     1     2     3     4     5     6     7     8     9    10    11    12    13    14    15    16    17    18    19 ... (array truncado)
B-GPU:     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0 ... (array truncado)
B-CPU: 32896 33153 33411 33670 33930 34191 34453 34716 34980 35245 35511 35778 36046 36315 36585 36856 37128 37401 37675 37950 ... (array truncado)
Resultados diferentes



In [5]:
%%gpu
#include<stdio.h>
#include<cuda.h>


#define BLOCKSIZE 1024  //threads por bloco
#define RANGE 256       //deslocamento utilizado para cálculo de B

void initArray(int *a, int n) {
    for (int i =0; i<n; ++i)
        a[i]=i;
}

void printArray(int *a, int n) {
    int maxPrint=20;
    for (int i =0; i< min(n,maxPrint); ++i)
        printf("%5d ",a[i]);
    if (n>maxPrint) printf("... (array truncado)");
    printf("\n");
}

bool checkArray(int *a, int *b, int n) {
    for (int i =0; i<n; ++i)
        if (a[i]!=b[i])
            return false;
    return true;
}

void execHost(int *A, int n, int *B) {
    for(int i=0; i<n; i++) {
        int soma=0;
        for(int j=i-RANGE; j<=i+RANGE; j++)
            if(j >= 0 && j<n)
                soma+=A[j];
        B[i]=soma;
    }
}


__global__
void vecReadTest(int* A,int n) {
    int i=blockIdx.x * blockDim.x + threadIdx.x;
    if (i<n)
      printf("\nThread %d lendo A[%d] = %d\n", i, i, A[i]);
}


__global__
void vecReadRange(int* A,int n) {
    int i=blockIdx.x * blockDim.x + threadIdx.x;
    for(int j=i-RANGE; j<=i+RANGE; j++)
        if(j >= 0 && j<n)
          printf("Bloco %d, Thread %d lendo A[%d] = %d\n", blockIdx.x, threadIdx.x, j, A[j]);

}





__global__
void readIntervalRange(int* A,int n) {
    int i=blockIdx.x * blockDim.x + threadIdx.x;
    // Definir o intervalo de leitura
    int startIdx = max(0, i - RANGE);   // Limite inferior do intervalo
    int endIdx = min(n - 1, i + RANGE); // Limite superior do intervalo

    // A thread lê os índices que são múltiplos de blockDim.x dentro do intervalo
    for (int j = startIdx; j <= endIdx; j++) {
        if ((j - i) % blockDim.x == 0) {  // Verifica se j é múltiplo de blockDim.x a partir de i
            printf("Bloco %d, Thread %d lendo A[%d] = %d\n", blockIdx.x, threadIdx.x, j, A[j]);
        }
    }

}



__global__
void execDevice_LENTO(int* A, int *B, int n) {
    // Cálculo do índice global
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    
    

   //várias comparações desnecessárias ao calcular os limites do intervalo para cada thread.
    if (i < n) { // Verifica se o índice está dentro do tamanho do vetor
        // Define os limites do intervalo
        int startIdx = max(0, i - RANGE);   // Limite inferior do intervalo
        int endIdx = min(n - 1, i + RANGE); // Limite superior do intervalo
    
   

        // Inicializa B[i]
        B[i] = 0;

        // Soma os valores do intervalo no vetor A para a posição B[i]
        for (int j = startIdx; j <= endIdx; j++) {
            B[i] += A[j];
        }
    }
}



__global__
void execDevice(int* A, int *B, int n) {
    
     
    int shared_SIZE=BLOCKSIZE + 2 * RANGE;
    __shared__ int sharedVec[BLOCKSIZE + 2 * RANGE];

    int globalIdx = blockIdx.x * blockDim.x + threadIdx.x; // Índice global
    int localIdx = threadIdx.x; // Índice local da thread no bloco

    // Índice inicial e final no vetor global A para este bloco
    int startIdx = blockIdx.x * blockDim.x - RANGE;
    int endIdx = (blockIdx.x + 1) * blockDim.x + RANGE - 1;



    //primeiro, preenche o próprio valor no vetor shared
    if(globalIdx>=0 && globalIdx<n){
        sharedVec[localIdx+RANGE]=A[globalIdx];
    }else{
        sharedVec[localIdx+RANGE]=0;
    }
    
    // Preenche a memória compartilhada com os valores apropriados ou 0
    //Se os valores à esquerda não existem no vetor A
    //Preenche todos os valores à esquerda do vetor shared
    if(globalIdx-RANGE<0){
        sharedVec[localIdx]=0;
    }else{
        sharedVec[localIdx]=A[globalIdx-RANGE];
    }

    //Se os valores à direita não existem no vetor A
    //Preenche todos os valores à direita do vetor shared
    if(globalIdx+RANGE>=n){
        sharedVec[localIdx+RANGE+RANGE]=0;
    }else{
        sharedVec[localIdx+RANGE+RANGE]=A[globalIdx+RANGE];
    }

    
    //Deve esperar todas as threads chegarem para que o shared esteja *completo*;
    __syncthreads(); // Sincroniza todas as threads do bloco


    /*
    for(int i =0; i<shared_SIZE; i++){
        printf("Bloco %d SharedA[%d] = %d \n",blockIdx.x,i,sharedVec[i]);
    }*/


    if(globalIdx>=0 && globalIdx<n){
      int soma=0;
      //Leitura do shared
      for(int i=0; i<(RANGE*2)+1;i++){
         //sharedVec[localIdx+i]
         //printf("Bloco %d Thread %d SharedA[%d] = %d \n",blockIdx.x,threadIdx.x,i,sharedVec[localIdx+i]);
         soma+=sharedVec[localIdx+i];
      }
      B[globalIdx]=soma;
      }
                              
    
    
    

   
}
int main() {
    int N = 1<<20;              //tamanho dos arrays A e B
    //int N=12;
    int *a = new int[N];
    int *b_cpu = new int[N];    //array B calculado na CPU
    int *b_gpu = new int[N];    //array B calculado na GPU


    initArray(a,N);
    printf("    A: ");
    printArray(a,N);

    int *d_a, *d_b;

    // Alocação de memória para o vetor A e B no dispositivo
    cudaError_t err;
    int size=N * sizeof(int);
    err = cudaMalloc((void**)&d_a,size );
    if (err != cudaSuccess) {
        printf("Erro na alocação de memória para A: %s\n", cudaGetErrorString(err));
        return -1;
    }

    err = cudaMalloc((void**)&d_b, size);
    if (err != cudaSuccess) {
        printf("Erro na alocação de memória para B: %s\n", cudaGetErrorString(err));
        return -1;
    }

    // Copiar os dados de A da CPU para o dispositivo
    err = cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    if (err != cudaSuccess) {
        printf("Erro na cópia de memória de A para o dispositivo: %s\n", cudaGetErrorString(err));
        return -1;
    }

    printf("B-GPU: ");
    
    // Tamanho da memória compartilhada
    int sharedMemSize = (BLOCKSIZE + 2 * RANGE) * sizeof(int); // Tamanho da memória compartilhada
    int numBlocks = (N + BLOCKSIZE - 1) / BLOCKSIZE;

    // Execução do kernel
    execDevice<<<numBlocks, BLOCKSIZE, sharedMemSize>>>(d_a, d_b, N);
    
    // Verifique se houve algum erro durante a execução do kernel
    err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("Erro na execução do kernel: %s\n", cudaGetErrorString(err));
        return -1;
    }

    // Copiar os resultados de volta da GPU para a CPU
    err = cudaMemcpy(b_gpu, d_b, size, cudaMemcpyDeviceToHost);
    if (err != cudaSuccess) {
        printf("Erro na cópia de memória de B para a CPU: %s\n", cudaGetErrorString(err));
        return -1;
    }

    printArray(b_gpu, N);

    printf("B-CPU: ");
    execHost(a, N, b_cpu);
    printArray(b_cpu, N);

    if (checkArray(b_gpu, b_cpu, N))
        printf("Resultados iguais\n");
    else
        printf("Resultados diferentes\n");

    // Liberar memória
    cudaFree(d_a);
    cudaFree(d_b);
    delete[] a;
    delete[] b_cpu;
    delete[] b_gpu;

    return 0;
}

    A:     0     1     2     3     4     5     6     7     8     9    10    11 
B-GPU:     6    10    15    21    28    35    42    49    56    51    45    38 
B-CPU:     6    10    15    21    28    35    42    49    56    51    45    38 
Resultados iguais

